In [6]:
%cd C:/Users/alapa/funiegan/

C:\Users\alapa\funiegan


C:\Users\alapa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
from piqa import PSNR
import torch
import cv2
import numpy as np

In [ ]:
# Load images (ensure they have the same dimensions)
original = 'C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainB/264286_00007889.jpg'
#original = cv2.imread('C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x2plus.pth/output__264286_00007889.jpg')
processed = cv2.imread('C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x2.pth/output_264286_00007889.jpg')  # Ground truth
# processed = cv2.imread('EUVP/test_samples/result/metricsevaluation/SA(batch8)/underwater_dark/generator_19/264286_00007889.jpg')  # Model output
# processed = cv2.imread('EUVP/test_samples/result/metricsevaluation/SA(batch3)/underwater_dark/generator_3/264286_00007889.jpg')  # Model output
# processed = cv2.imread('EUVP/test_samples/result/metricsevaluation/noattention/underwater_dark/generator_8/264286_00007889.jpg')  # Model output

# Convert to float32 and normalize to [0, 1]
original = original.astype(np.float32) / 255.0
processed = processed.astype(np.float32) / 255.0

# Convert to PyTorch tensors (add batch and channel dimensions)
original_tensor = torch.from_numpy(original).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]
processed_tensor = torch.from_numpy(processed).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]

# Compute PSNR
psnr = PSNR()
psnr_value = psnr(processed_tensor, original_tensor)
print(f"PSNR: {psnr_value.item()} dB")

PSNR: 21.01607322692871 dB


In [41]:
from skimage.measure import shannon_entropy
import cv2
import numpy as np

def compute_entropy(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    
    # Compute entropy
    entropy_value = shannon_entropy(image)
    return entropy_value

# Example usage
image_path = "C:/Users/alapa/funiegan/EUVP/test_samples/result/metricsevaluation/SA(batch8)/underwater_dark/generator_19/264286_00007889.jpg"
# image_path = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainB/264286_00007889.jpg"
# image_path = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x4.pth/output_264286_00007889.jpg"  # Replace with your image path
try:
    entropy_value = compute_entropy(image_path)
    print(f"Entropy of the image: {entropy_value}")
except FileNotFoundError as e:
    print(e)

Entropy of the image: 7.206697756614872


In [ ]:
import cv2
import numpy as np

def compute_uicm(image):
    """
    Compute UICM (Underwater Image Colorfulness Measure).
    """
    r, g, b = cv2.split(image.astype(np.float32))
    rg = r - g
    yb = (r + g) / 2 - b
    rg_std = np.std(rg)
    yb_std = np.std(yb)
    uicm = np.sqrt(rg_std**2 + yb_std**2)
    return uicm

def compute_uism(image):
    """
    Compute UISM (Underwater Image Sharpness Measure).
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    uism = np.mean(np.abs(laplacian))
    return uism

def compute_uiconm(image):
    """
    Compute UIConM (Underwater Image Contrast Measure).
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(gray.ravel(), bins=256, range=(0, 256))
    prob = hist / hist.sum()
    prob = prob[prob > 0]  # Avoid log(0)
    uiconm = -np.sum(prob * np.log2(prob))  # Shannon entropy
    return uiconm

def compute_uiqm(image_path):
    """
    Compute UIQM (Underwater Image Quality Measure).
    Args:
        image_path (str): Path to the input image.
    Returns:
        float: UIQM value.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Compute components
    uicm = compute_uicm(image)
    uism = compute_uism(image)
    uiconm = compute_uiconm(image)

    # Compute UIQM
    uiqm = 0.0282 * uicm + 0.2953 * uism + 3.5753 * uiconm
    return uiqm

# Example usage

# image_path = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainA/264286_00007889.jpg"  
# image_path = "C:/Users/alapa/funiegan/EUVP/test_samples/result/metricsevaluation/SA(batch8)/underwater_dark/generator_19/264286_00007889.jpg"
# image_path = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainB/264286_00007889.jpg"
# image_path = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x8.pth/output_264286_00007889.jpg" 
try:
    uiqm_value = compute_uiqm(image_path)
    print(f"UIQM: {uiqm_value}")
except FileNotFoundError as e:
    print(e)

UIQM: 28.479959846866834
